In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
print(check_output(["ls", "../input/video_level"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

In [ ]:
labels_df = pd.read_csv('../input/label_names.csv')
#filenames = ["../input/video_level/train-{}.tfrecord".format(i) for i in range(10)]
print("we have {} unique labels in the dataset".format(len(labels_df['label_name'].unique())))

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
labels_df = pd.read_csv('../input/label_names.csv')
labels = []
textual_labels = []
textual_labels_nested = []
filenames = ["../input/video_level/train-{}.tfrecord".format(i) for i in range(10)]
total_sample_counter = 0

label_counts = []

for filename in filenames:
    for example in tf.python_io.tf_record_iterator(filename):
        total_sample_counter += 1
        tf_example = tf.train.Example.FromString(example)

        label_example = list(tf_example.features.feature['labels'].int64_list.value)
        label_counts.append(len(label_example))
        labels = labels + label_example
        label_example_textual = list(labels_df[labels_df['label_id'].isin(label_example)]['label_name'])
        textual_labels_nested.append(set(label_example_textual))
        textual_labels = textual_labels + label_example_textual
        if len(label_example_textual) != len(label_example):
            print('label names lookup failed: {} vs {}'.format(label_example, label_example_textual))

print('label ids missing from label_names.csv: {}'.format(len(sorted(set(labels) - set(labels_df['label_id'])))))
print('Found {} samples in all of the 10 available tfrecords'.format(total_sample_counter))

In [ ]:
def grouped_data_for(l):
    l_with_c = pd.DataFrame(
        pd.DataFrame({'label': l}).groupby('label').size().rename('n')
    ).sort_values('n', ascending=False).reset_index()
    return l_with_c

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.distplot(label_counts, kde=False)
plt.title('distribution of number of labels')

In [ ]:
N = 20

textual_labels_with_counts_all = grouped_data_for(textual_labels)

sns.barplot(y='label', x='n', data=textual_labels_with_counts_all.iloc[0:N, :])
plt.title('Top {} labels with sample count'.format(N))

In [ ]:
import tensorflow as tf
labels_df = pd.read_csv('../input/label_names.csv')
filenames = ["../input/video_level/train-{}.tfrecord".format(i) for i in range(10)]

cosmetics = []
games = []
car = []
vehicle = []
animal = []

for filename in filenames:
    for example in tf.python_io.tf_record_iterator(filename):
        tf_example = tf.train.Example.FromString(example)
        label_example = list(tf_example.features.feature['labels'].int64_list.value)
        label_example_textual = list(labels_df[labels_df['label_id'].isin(label_example)]['label_name'])
        mean_mean_rgb = np.mean(tf_example.features.feature['mean_rgb'].float_list.value)
        for label in label_example_textual:
            if label == 'Cosmetics':
                cosmetics.append(mean_mean_rgb)
            elif label == 'Games':
                games.append(mean_mean_rgb)
            elif label == 'Car':
                car.append(mean_mean_rgb)
            elif label == 'Vehicle':
                vehicle.append(mean_mean_rgb)
            elif label == 'Animal':
                animal.append(mean_mean_rgb)